# The Laplace approximation on GLMM w/ full parameter space estimated

## Import dependents

In [1]:
from numpy.linalg import inv
import pandas as pd
import numpy as np
import time
from scipy.stats import norm
from scipy.linalg import block_diag

## Definitions

- All inputs follows structure: [$x$, $y$, $\mu$, $\beta$, $\tau$]

Log-Sum_exp trick:

In [2]:
def logsumexp(x):
    c = max(x)
    LSE = c + np.log(np.sum(np.exp(x - c)))
    return np.exp(x - LSE)

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta}+\tau_{i}\mu_{i})}{1 + \exp{(X_{ij}^\top\beta}+\tau_{i}\mu_{i})}$$

In [3]:
def Pi(x, y, mu, beta, tau):
    result = np.asarray((np.exp(x @ beta + tau * mu) / (1 + np.exp(x @ beta + tau * mu))))
    # Check if there are exp(0) cases, if true, return \pi = 1 correspondingly
    return np.nan_to_num(result, nan = 1)

#     if np.exp(x @ beta + mu).max() == np.inf:
#         return np.nan_to_num(result, nan = 1)
#     else:
#         return result

Notation for $g(\mu_{i})$:

$$g(\mu_{i};\beta)=\sum_{j=1}^{n_i}\left[y_{ij}\log\pi_{ij}+(1-y_{ij})\log(1-\pi_{ij})\right]+\log[\tau_{i}\phi(\mu_{i},1)]$$

In [4]:
def g(x, y, mu, beta, tau):
    g = sum(y * logsumexp(Pi(x, y, mu, beta, tau)) + (1 - y) * logsumexp(1 - Pi(x, y, mu, beta, tau))) \
    + np.nan_to_num(np.log(tau * (np.sqrt(2 * np.pi))**(-1) * np.exp(-mu**2/2)), nan=0)
    return g

### 1. First derivative

Notation for $g_\mu(\mu_{i})$:

$$\dfrac{\partial g}{\partial \mu_{i}}=\sum_{j=1}^{n_i}\tau_{i}(y_{ij}-\pi_{ij})-\mu_{i}$$

In [5]:
def g_u(x, y, mu, beta, tau):
    return tau * sum((y - Pi(x, y, mu, beta, tau))) - mu

Notation for $g_\beta(\beta)$:

$$\dfrac{\partial g}{\partial \beta}=\sum_{j=1}^{n_i}X_{ij}(y_{ij}-\pi_{ij})$$

In [6]:
def g_b(x, y, mu, beta, tau):
    return np.sum(x * y - x * Pi(x, y, mu, beta, tau), axis = 0)

Notation for $g_\tau(\tau_{i})$:

$$\dfrac{\partial g}{\partial \tau_{i}}=\sum_{j=1}^{n_i}\mu_{i}(y_{ij}-\pi_{ij})$$

In [7]:
def g_t(x, y, mu, beta, tau):
    return mu * sum((y - Pi(x, y, mu, beta, tau)))

### 2. Second derivative

Notation for $g_{\mu\mu}(\mu_{i})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i}^2}=-\tau_i\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_{i}}-1$$

In [8]:
def g_uu(x, y, mu, beta, tau):
    result = np.nan_to_num(tau * np.asarray((np.exp(x @ beta + tau * mu) /\
                                              (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return - tau * sum(result) - 1

Notation for $g_{\beta\beta}(\beta)$:

$$\dfrac{\partial^2 g}{\partial \beta^2}=-\sum_{j=1}^{n_i}X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta}$$

In [9]:
def g_bb(x, y, mu, beta, tau):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
        * np.nan_to_num((np.exp(x[i] @ beta + tau * mu) / (1 + np.exp(x[i] @ beta + tau * tau * mu))**2), nan = 0))
    return result

Notation for $g_{\tau\tau}(\tau_{i})$:

$$\dfrac{\partial^2 g}{\partial \tau_i^2}=-\mu_i\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\tau_{i}}$$

In [10]:
def g_tt(x, y, mu, beta, tau):
    result = np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                              (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return -mu * sum(result)

Notation for $g_{\mu\beta}(\beta)$:

$$\dfrac{\partial^2 g}{\partial \mu_{i}\partial \beta}=-\tau_i\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\beta}$$

In [11]:
def g_ub(x, y, mu, beta, tau):
    result = np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) /\
                                       (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return -tau * np.sum(x * result, axis = 0)

Notation for $g_{\mu\tau}(\tau_i)$:

$$\dfrac{\partial^2 g}{\partial\mu_i\partial\tau_i}=\sum_{j=1}^{n_i}\left(y_{ij}-\pi_{ij}\right)-\tau_i\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\tau_i}$$

In [12]:
def g_ut(x, y, mu, beta, tau):
    result = sum(y - Pi(x, y, mu, beta, tau))\
    - tau * sum(mu * np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                          (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0))
    return result

Notation for $g_{\beta\tau}(\beta,\tau)$:

$$\dfrac{\partial^2 g}{\partial \beta\partial \tau_i}=-\sum_{j=1}^{n_i}X_{ij}\dfrac{\partial\pi_{ij}}{\partial\tau_i}$$

In [13]:
def g_bt(x, y, mu, beta, tau):
    result = np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                          (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return -np.sum(x * result, axis = 0)

### 3. Third derivative

Notation for $g_{\mu\mu\mu}(\hat\mu_i,\beta)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i}^3} = -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i}^2}$$

In [14]:
def g_uuu(x, y, mu, beta, tau):
    return -tau * sum(-tau**2 * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0))

Notation for $g_{\mu\mu\beta}(\hat\mu_{i},\beta)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i}^2\partial \beta} = -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i}\partial\beta}$$

In [15]:
def g_uub(x, y, mu, beta, tau): # add -
    return -tau * np.sum(-tau * x * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0), axis = 0)

Notation for $g_{\mu\beta\beta}(\hat\mu_{i},\beta)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i}\partial \beta^2}=-\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\beta^2}$$

In [16]:
def g_ubb(x, y, mu, beta, tau):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                         * np.nan_to_num((np.exp(x[i] @ beta + tau * mu) * (np.exp(x[i] @ beta + tau * mu) - 1)\
                                       / (1 + np.exp(x[i] @ beta + tau * mu))**3), nan = 0))
    return -tau * result

Notation for $g_{\mu\mu\tau}(\hat\mu_{i},\beta)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i}^2\partial \tau_i} = -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i}\partial\tau_i}-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_i}$$

In [17]:
def g_uut(x, y, mu, beta, tau):
    result = -tau * np.sum(np.nan_to_num((np.exp(x @ beta + tau * mu) * ((1 - tau * mu) * np.exp(x @ beta + tau * mu) + tau * mu + 1))\
    / (1 + np.exp(x @ beta + tau * mu))**3, nan = 0), axis = 0) - np.sum(np.nan_to_num(tau * np.asarray((np.exp(x @ beta + tau * mu) /\
                                              (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0))
    return result

Notation for $g_{\mu\tau\tau}(\hat\mu_{i},\beta)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i}\partial \tau_i^2} = -2\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\tau_i}-\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\tau_i^2}$$

In [18]:
def g_utt(x, y, mu, beta, tau):
    result = -2 * np.sum(np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                          (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0), axis = 0)\
    - tau * np.sum( np.nan_to_num(-mu**2 * np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1) /\
                   (1 + np.exp(x @ beta + tau * mu))**3, nan = 0) )
    return result

Notation for $g_{\mu\beta\tau_{i}}(\hat\mu_{i},\beta, \tau_i)$:

$$
\dfrac{\partial^3 g}{\partial \mu_{i}\partial \beta\partial\tau_i}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\beta} - \tau_i\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\beta\partial\tau_i}
$$

In [19]:
def g_ubt(x, y, mu, beta, tau):
    left = np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) /\
                                       (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    left = -np.sum(x * left, axis = 0)
    right = np.nan_to_num(np.asarray((mu * np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1))/\
                                    (np.exp(x @ beta + tau * mu) + 1)**3), nan = 0)
    right = -np.sum(x * right, axis = 0)
    return left + right

### 4. Forth derivative

Notation for $g_{\mu\mu\mu\mu}(\hat\mu_{i},\beta_0)$:

$$g_{\mu\mu\mu\mu}(\hat\mu_{i},\beta) = -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i}^3}$$

In [20]:
def g_uuuu(x, y, mu, beta, tau):
    result = -tau * np.sum( np.nan_to_num(-tau**3 * np.exp(2 * x @ beta + 2 * tau * mu)/(np.exp(x @ beta + tau * mu) + 1)**3\
                           - tau**3 * np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)/(np.exp(x @ beta + tau * mu) + 1)**3\
                           + 3 * tau**3 * np.exp(2 * x @ beta + 2 * tau * mu) * (np.exp(x @ beta + tau * mu) - 1)/(np.exp(x @ beta + tau * mu) + 1)**4, nan = 0) )
    return result

In [21]:
# def g_uuuu(x, y, mu, beta, tau):
#     result = sum(- np.nan_to_num((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1) \
#                                   / (1 + np.exp(x @ beta + tau * mu)) ** 3), nan=0) \
#                  + np.nan_to_num((3 * np.exp(2 * (x @ beta + tau * mu)) * (np.exp(x @ beta + tau * mu) - 1) \
#                                   / (1 + np.exp(x @ beta + tau * mu)) ** 4), nan=0) \
#                  - np.nan_to_num((np.exp(2 * (x @ beta + tau * mu)) / (1 + np.exp(x @ beta + tau * mu)) ** 3), nan=0) \
#                  )
#     return result

Notation for $g_{\mu\mu\mu\beta}(\hat\mu_{i},\beta)$:

$$g_{\mu\mu\mu\beta}(\hat\mu_{i},\beta) = -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i}^2\partial\beta}$$

In [22]:
def g_uuub(x, y, mu, beta, tau):
    result = -tau * np.sum( np.nan_to_num(tau**2 * x * np.exp(x @ beta + tau * mu)\
                           * (-4 * np.exp(x @ beta + tau * mu) + np.exp(2 * x @ beta + 2 * tau * mu) + 1)\
                           /(np.exp(x @ beta + tau * mu) + 1)**4, nan = 0), axis = 0 )
    return result

In [23]:
# def g_uuub(x, y, mu, beta, tau):
#     result = tau * np.sum(- x * np.asarray(np.nan_to_num((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1) \
#                                                     / (1 + np.exp(x @ beta + tau * mu)) ** 3), nan=0) \
#                                      + np.nan_to_num(
#         (3 * np.exp(2 * (x @ beta + tau * mu)) * (np.exp(x @ beta + tau * mu) - 1) \
#          / (1 + np.exp(x @ beta + tau * mu)) ** 4), nan=0) \
#                                      - np.nan_to_num(
#         (np.exp(2 * (x @ beta + tau * mu)) / (1 + np.exp(x @ beta + tau * mu)) ** 3), nan=0))
#                     , axis=0)
#     return result

Notation for $g_{\mu\mu\beta\beta}(\hat\mu_{i},\beta)$:

$$g_{\mu\mu\beta\beta}(\hat\mu_{i},\beta) = -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i}\partial\beta^2}$$

In [24]:
def g_uubb(x, y, mu, beta, tau):
    result = 0
    for i in range(len(y)):
        result += x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1]) * np.nan_to_num(np.exp(x[i] @ beta + tau * mu)\
        * (-4 * np.exp(x[i] @ beta + tau * mu) + np.exp(2 * x[i] @ beta + 2 * tau * mu) + 1)\
        / (np.exp(x[i] @ beta + tau * mu) + 1)**4, nan=0)
        
    result *= tau                     
        
    return -tau * result

Notation for $g_{\mu\mu\mu\tau}(\hat\mu_{i},\beta)$:

$$g_{\mu\mu\mu\tau}(\hat\mu_{i},\beta) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_i^2} -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i}^2\partial\tau_i}$$

In [25]:
def g_uuut(x, y, mu, beta, tau):
    left = -np.sum(-tau**2 * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0))
    right = -tau * tau * np.sum(np.nan_to_num(np.exp(x @ beta + tau * mu) * (-4 * tau * mu * np.exp(x @ beta + tau * mu)\
                                                        + tau * mu * np.exp(2 * x @ beta + 2 * tau * mu)\
                                                        -2 * np.exp(2 * x @ beta + 2 * tau * mu) + tau * mu + 2)\
           /(np.exp(x @ beta + tau * mu) + 1)**4, nan=0))
    return left + right

Notation for $g_{\mu\mu\tau\tau}(\hat\mu_{i},\beta)$:

$$g_{\mu\mu\tau\tau}(\hat\mu_{i},\beta) = -2\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_i\partial\tau_i} -\tau_i\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i}\partial\tau_i^2}$$

In [26]:
def g_uutt(x, y, mu, beta, tau):
    left = -2 * np.sum(np.nan_to_num((np.exp(x @ beta + tau * mu)\
                                  * ((1 - tau * mu) * np.exp(x @ beta + tau * mu) + tau * mu + 1))\
    / (1 + np.exp(x @ beta + tau * mu))**3, nan = 0), axis = 0)
    right = -tau * mu * np.sum(np.nan_to_num(np.exp(x @ beta + tau * mu) * (-4 * tau * mu * np.exp(x @ beta + tau * mu)\
                                                        + tau * mu * np.exp(2 * x @ beta + 2 * tau * mu)\
                                                        -2 * np.exp(2 * x @ beta + 2 * tau * mu) + tau * mu + 2)\
           /(np.exp(x @ beta + tau * mu) + 1)**4, nan=0))
    return left + right

Notation for $g_{\mu\mu\beta\tau_{i}}(\hat\mu_{i},\beta, \tau_i)$:

$$
g_{\mu\mu\beta\tau}=-\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i}\partial\beta} - \tau_i\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i}\partial\beta\partial\tau_i}
$$

In [27]:
def g_uubt(x, y, mu, beta, tau):
    left = -np.sum(-tau * x * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0), axis = 0)
    right = -tau * np.sum(x * np.nan_to_num(np.asarray(np.exp(x @ beta + tau * mu) * (-4 * tau * mu * np.exp(x @ beta + tau * mu)\
                                                             + tau * mu * np.exp(2 * x @ beta + 2 * tau * mu)\
                                                             - np.exp(2 * x @ beta + 2 * tau * mu) + tau * mu + 1)\
                          / (np.exp(x @ beta + tau * mu) + 1)**4), nan = 0), axis = 0)
    return left + right

### 5. Other derivatives

Notation for $\hat\omega$:

$$\hat\omega=\sqrt{-\dfrac{1}{g_{\mu\mu}(\hat\mu_{i}, \theta)}}$$

In [28]:
def omega(x, y, mu, beta, tau):
    return np.sqrt(-1/g_uu(x, y, mu, beta, tau))

Notation for $\hat\mu_\beta$

$$\hat\mu_\beta(\beta)=-\dfrac{g_{\mu\beta}(\hat\mu_{i},\beta)}{g_{\mu\mu}(\hat\mu_{i},\beta)}=\hat\omega^2(\beta)g_{\mu\beta}(\hat\mu_{i}(\beta),\beta)$$

In [29]:
def mu_b(x, y, mu, beta, tau):
    return omega(x, y, mu, beta, tau)**2 * g_ub(x, y, mu, beta, tau)

Notation for $\hat\mu_{\beta\beta'}$:

$$\hat\mu_{\beta\beta'}(\beta)=\hat\omega^2(\hat\mu_\beta\hat\mu_{\beta'} g_{\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\beta'}+\hat\mu_{\beta'}g_{\mu\mu\beta}+g_{\mu\beta\beta'})$$

In [30]:
def mu_bb(x, y, mu, beta, tau):
    result = omega(x, y, mu, beta, tau)**2 * (mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
                                              @ mu_b(x, y, mu, beta, tau).reshape(1,x.shape[1])\
                                              * g_uuu(x, y, mu, beta, tau)\
                                              + 2 * mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
                                              @ g_uub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
                                              + g_ubb(x, y, mu, beta, tau))
    return result

Notation for $\hat\omega_\beta$:

$$\dfrac{\partial\hat\omega}{\partial\beta}=\dfrac{1}{2}\hat\omega^3(g_{\mu\mu\mu}\hat\mu_\beta + g_{\mu\mu\beta})$$

In [31]:
def omega_b(x, y, mu, beta, tau):
    return 0.5 * omega(x, y, mu, beta, tau)**3 * (g_uuu(x, y, mu, beta, tau) * mu_b(x, y, mu, beta, tau)\
                                                    + g_uub(x, y, mu, beta, tau))

Notation for $\hat\mu_\tau$

$$\hat\mu_\tau(\tau)=-\dfrac{g_{\mu\tau}(\hat\mu_{i},\tau)}{g_{\mu\mu}(\hat\mu_{i},\tau)}=\hat\omega^2(\tau)g_{\mu\tau}(\hat\mu_{i}(\tau),\tau)$$

In [32]:
def mu_t(x, y, mu, beta, tau):
    return omega(x, y, mu, beta, tau)**2 * g_ut(x, y, mu, beta, tau)

Notation for $\hat\mu_{\tau\tau'}$:

$$\hat\mu_{\tau\tau'}(\tau)=\hat\omega^2(\hat\mu_\tau\hat\mu_{\tau'} g_{\mu\mu\mu}+\hat\mu_{\tau}g_{\mu\mu\tau'}+\hat\mu_{\tau'}g_{\mu\mu\tau}+g_{\mu\tau\tau'})$$

In [33]:
def mu_tt(x, y, mu, beta, tau):
    result = omega(x, y, mu, beta, tau)**2 * (mu_t(x, y, mu, beta, tau)\
                                              * mu_t(x, y, mu, beta, tau)\
                                              * g_uuu(x, y, mu, beta, tau)\
                                              + 2 * mu_t(x, y, mu, beta, tau)\
                                              * g_uut(x, y, mu, beta, tau)\
                                              + g_utt(x, y, mu, beta, tau))
    return result

Notation for $\hat\mu_{\beta\tau}$:

$$\hat\mu_{\beta\tau}(\beta)=\hat\omega^2(\hat\mu_\beta\hat\mu_{\tau} g_{\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\tau}+\hat\mu_{\tau}g_{\mu\mu\beta}+g_{\mu\beta\tau})$$

In [34]:
def mu_bt(x, y, mu, beta, tau):
    result = omega(x, y, mu, beta, tau)**2 * (mu_b(x, y, mu, beta, tau)\
                                              * mu_t(x, y, mu, beta, tau)\
                                              * g_uuu(x, y, mu, beta, tau)\
                                              + mu_b(x, y, mu, beta, tau)\
                                              * g_uut(x, y, mu, beta, tau)\
                                              + mu_t(x, y, mu, beta, tau)\
                                              * g_uub(x, y, mu, beta, tau)\
                                              + g_ubt(x, y, mu, beta, tau))
    return result

Notation for $\hat\omega_\tau$:

$$\dfrac{\partial\hat\omega}{\partial\tau}=\dfrac{1}{2}\hat\omega^3(g_{\mu\mu\mu}\hat\mu_\tau + g_{\mu\mu\tau})$$

In [35]:
def omega_t(x, y, mu, beta, tau):
    return 0.5 * omega(x, y, mu, beta, tau)**3 * (g_uuu(x, y, mu, beta, tau) * mu_t(x, y, mu, beta, tau)\
                                                    + g_uut(x, y, mu, beta, tau))

Notation for $\hat\omega_{\beta\beta'}$:

$$\dfrac{\partial^2}{\partial\beta\partial\beta'}\hat\omega(\beta)=\frac{3}{4}\hat\omega^5(\hat\mu_{\beta'}g_{\mu\mu\mu}+g_{\mu\mu\beta'})(\hat\mu_{\beta}g_{\mu\mu\mu}+g_{\mu\mu\beta})+\dfrac{1}{2}\hat\omega^3(\hat\mu_{\beta\beta'}g_{\mu\mu\mu}+\hat\mu_\beta\hat\mu_{\beta'}g_{\mu\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\mu\beta'}+\hat\mu_{\beta'}g_{\mu\mu\mu\beta}+g_{\mu\mu\beta\beta'})$$

In [36]:
def omega_bb(x, y, mu, beta, tau):
    result = 3/4 * omega(x, y, mu, beta, tau)**5 * (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
                                                      + g_uub(x, y, mu, beta, tau)).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + g_uub(x, y, mu, beta, tau)).reshape(1,x.shape[1]) + 1/2 * omega(x, y, mu, beta, tau)**3\
     * (mu_bb(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + (mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
           @ mu_b(x, y, mu, beta, tau).reshape(1,x.shape[1]) * g_uuuu(x, y, mu, beta, tau))\
        + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
        + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
        + g_uubb(x, y, mu, beta, tau)
       )
    return result

In [37]:
# def omega_bb(x, y, mu, beta, tau):
#     result = 3 / 4 * omega(x, y, mu, beta, tau) ** 5 * (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau) \
#                                                           + g_uub(x, y, mu, beta, tau)).reshape(x.shape[1], 1) \
#              @ (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau) \
#                 + g_uub(x, y, mu, beta, tau)).reshape(1, x.shape[1]) + 1 / 2 * omega(x, y, mu, beta, tau) ** 3 \
#              * (mu_bb(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau) \
#                 + (mu_b(x, y, mu, beta, tau).reshape(x.shape[1], 1) \
#                    @ mu_b(x, y, mu, beta, tau).reshape(1, x.shape[1]) * g_uuuu(x, y, mu, beta, tau)) \
#                 + mu_b(x, y, mu, beta, tau).reshape(x.shape[1], 1) @ g_uuub(x, y, mu, beta, tau).reshape(1,
#                                                                                                              x.shape[
#                                                                                                                  1]) \
#                 + mu_bb(x, y, mu, beta, tau)
#                 )
#     return result

Notation for $\hat\omega_{\tau\tau'}$:

$$\dfrac{\partial^2}{\partial\tau\partial\tau'}\hat\omega(\tau)=\frac{3}{4}\hat\omega^5(\hat\mu_{\tau'}g_{\mu\mu\mu}+g_{\mu\mu\tau'})(\hat\mu_{\tau}g_{\mu\mu\mu}+g_{\mu\mu\tau})+\dfrac{1}{2}\hat\omega^3(\hat\mu_{\tau\tau'}g_{\mu\mu\mu}+\hat\mu_\tau\hat\mu_{\tau'}g_{\mu\mu\mu\mu}+\hat\mu_{\tau}g_{\mu\mu\mu\tau'}+\hat\mu_{\tau'}g_{\mu\mu\mu\tau}+g_{\mu\mu\tau\tau'})$$

In [38]:
def omega_tt(x, y, mu, beta, tau):
    result = 3/4 * omega(x, y, mu, beta, tau)**5 * (mu_t(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
                                                      + g_uut(x, y, mu, beta, tau))\
     * (mu_t(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + g_uut(x, y, mu, beta, tau)) + 1/2 * omega(x, y, mu, beta, tau)**3\
     * (mu_tt(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + (mu_t(x, y, mu, beta, tau)\
           * mu_t(x, y, mu, beta, tau) * g_uuuu(x, y, mu, beta, tau))\
        + mu_t(x, y, mu, beta, tau) * g_uuut(x, y, mu, beta, tau)\
        + mu_t(x, y, mu, beta, tau) * g_uuut(x, y, mu, beta, tau)\
        + g_uutt(x, y, mu, beta, tau)
       )
    return result

Notation for $\hat\omega_{\beta\tau}$:

$$\dfrac{\partial^2}{\partial\beta\partial\tau}\hat\omega(\beta)=\frac{3}{4}\hat\omega^5(\hat\mu_{\tau}g_{\mu\mu\mu}+g_{\mu\mu\tau})(\hat\mu_{\beta}g_{\mu\mu\mu}+g_{\mu\mu\beta})+\dfrac{1}{2}\hat\omega^3(\hat\mu_{\beta\tau}g_{\mu\mu\mu}+\hat\mu_\beta\hat\mu_{\tau}g_{\mu\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\mu\tau}+\hat\mu_{\tau}g_{\mu\mu\mu\beta}+g_{\mu\mu\beta\tau})$$

In [39]:
def omega_bt(x, y, mu, beta, tau):
    result = 3/4 * omega(x, y, mu, beta, tau)**5 * (mu_t(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
                                                      + g_uut(x, y, mu, beta, tau))\
     * (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + g_uub(x, y, mu, beta, tau)) + 1/2 * omega(x, y, mu, beta, tau)**3\
     * (mu_bt(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + (mu_b(x, y, mu, beta, tau)\
           * mu_t(x, y, mu, beta, tau) * g_uuuu(x, y, mu, beta, tau))\
        + mu_b(x, y, mu, beta, tau) * g_uuut(x, y, mu, beta, tau)\
        + mu_t(x, y, mu, beta, tau) * g_uuub(x, y, mu, beta, tau)\
        + g_uubt(x, y, mu, beta, tau)
       )
    return result

Notation for $\mathcal l_i'(\beta)$:

$$\mathcal l_i'(\beta) = \dfrac{\partial\mathcal l_i}{\partial\beta}=\dfrac{\hat\omega_\beta}{\hat\omega}+g_\mu(\hat\mu_{i},\beta)\hat\mu_\beta(\beta)+g_\beta(\hat\mu_{i},\beta)$$

In [40]:
def lb_1(x, y, mu, beta, tau):
    l1 = omega_b(x, y, mu, beta, tau) / omega(x, y, mu, beta, tau)\
    + g_u(x, y, mu, beta, tau) * mu_b(x, y, mu, beta, tau) + g_b(x, y, mu, beta, tau)
    return l1

Notation for $\mathcal l_i'(\tau)$:

$$\mathcal l_i'(\tau) = \dfrac{\partial\mathcal l_i}{\partial\tau}=\dfrac{\hat\omega_\tau}{\hat\omega}+g_\mu(\hat\mu_{i},\tau)\hat\mu_\tau(\tau)+g_\tau(\hat\mu_{i},\tau)$$

In [41]:
def lt_1(x, y, mu, beta, tau):
    l1 = omega_t(x, y, mu, beta, tau) / omega(x, y, mu, beta, tau)\
    + g_u(x, y, mu, beta, tau) * mu_t(x, y, mu, beta, tau) + g_t(x, y, mu, beta, tau)
    return l1

Notation for $l''_i(\beta)$:

$$l''_i(\beta)=\dfrac{\partial^2l_i}{\partial\beta^2}=\dfrac{\hat\omega_{\beta\beta'}\hat\omega-\hat\omega_\beta\hat\omega_{\beta'}}{\hat\omega^2}+\hat\mu_{\beta\beta'}g_\mu+\hat\mu_\beta(\hat\mu_{\beta'}g_{\mu\mu}+g_{\mu\beta'})+\hat\mu_{\beta'}g_{\mu\beta}+g_{\beta\beta'}$$

In [42]:
def lb_2(x, y, mu, beta, tau):
    l2 = omega(x, y, mu, beta, tau)**(-2) * (omega_bb(x, y, mu, beta, tau) * omega(x, y, mu, beta, tau)\
                                               - omega_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
                                               @ omega_b(x, y, mu, beta, tau).reshape(1,x.shape[1]))\
     + mu_bb(x, y, mu, beta, tau) * g_u(x, y, mu, beta, tau) + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta, tau) * g_uu(x, y, mu, beta, tau) + g_ub(x, y, mu, beta, tau)).reshape(1,x.shape[1])\
     + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1) @ g_ub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
     + g_bb(x, y, mu, beta, tau)
    return l2

Notation for $l''_i(\tau)$:

$$l''_i(\tau)=\dfrac{\partial^2l_i}{\partial\tau^2}=\dfrac{\hat\omega_{\tau\tau'}\hat\omega-\hat\omega_\tau\hat\omega_{\tau'}}{\hat\omega^2}+\hat\mu_{\tau\tau'}g_\mu+\hat\mu_\tau(\hat\mu_{\tau'}g_{\mu\mu}+g_{\mu\tau'})+\hat\mu_{\tau'}g_{\mu\tau}+g_{\tau\tau'}$$

In [43]:
def lt_2(x, y, mu, beta, tau):
    l2 = omega(x, y, mu, beta, tau)**(-2) * (omega_tt(x, y, mu, beta, tau) * omega(x, y, mu, beta, tau)\
                                               - omega_t(x, y, mu, beta, tau)\
                                               * omega_t(x, y, mu, beta, tau))\
     + mu_tt(x, y, mu, beta, tau) * g_u(x, y, mu, beta, tau) + mu_t(x, y, mu, beta, tau)\
     * (mu_t(x, y, mu, beta, tau) * g_uu(x, y, mu, beta, tau) + g_ut(x, y, mu, beta, tau))\
     + mu_t(x, y, mu, beta, tau) * g_ut(x, y, mu, beta, tau)\
     + g_tt(x, y, mu, beta, tau)
    return l2

Notation for $l''_i(\beta, \tau)$:

$$l''_i(\beta)=\dfrac{\partial^2l_i}{\partial\beta\partial\tau_i}=\dfrac{\hat\omega_{\beta\tau}\hat\omega-\hat\omega_\beta\hat\omega_{\tau}}{\hat\omega^2}+\hat\mu_{\beta\tau}g_\mu+\hat\mu_\beta(\hat\mu_{\tau}g_{\mu\mu}+g_{\mu\tau})+\hat\mu_{\tau}g_{\mu\beta}+g_{\beta\tau}$$

In [44]:
def lbt_2(x, y, mu, beta, tau):
    l2 = omega(x, y, mu, beta, tau)**(-2) * (omega_bt(x, y, mu, beta, tau) * omega(x, y, mu, beta, tau)\
                                               - omega_b(x, y, mu, beta, tau)\
                                               * omega_t(x, y, mu, beta, tau))\
     + mu_bt(x, y, mu, beta, tau) * g_u(x, y, mu, beta, tau) + mu_b(x, y, mu, beta, tau)\
     * (mu_t(x, y, mu, beta, tau) * g_uu(x, y, mu, beta, tau) + g_ut(x, y, mu, beta, tau))\
     + mu_t(x, y, mu, beta, tau) * g_ub(x, y, mu, beta, tau)\
     + g_bt(x, y, mu, beta, tau)
    return l2

Notation for maximizing $\mu$ with respect to $\hat\mu=\arg\max_\mu g(\mu_{i0};\beta_0)$

In [45]:
def max_mu(x, y, mu, beta, tau, max_iter=100):
    for step in range(max_iter):
        mu_new = mu - g_u(x, y, mu, beta, tau)/g_uu(x, y, mu, beta, tau)
        diff = mu_new - mu
        mu = mu_new
        if np.abs(diff) < 10**(-10):
            break;       
    return mu

Notation for $l_i(\mu, \beta, \tau)$:

$$l_i(\mu, \beta, \tau) = \dfrac{1}{2}\log{(2\pi)}+\log(\hat\omega)+g(\hat\mu_{i},\beta,\tau)$$

In [46]:
def l(x, y, mu, beta, tau):
    l = 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta, tau)) + g(x, y, mu, beta, tau)
    return l

### Laplace approximation class

In [47]:
# class LA:
#     """
#     A calss to run distributed GLMM

#     ...

#     Attributes
#     ------------
#     X : A list of DataFrames
#         The data from different sites
#     y : A list of arrays
#         The binary outcomes from different sites
#     beta : An array of params
#         Fixed effects coefficents
#     lam : Float
#         Regularization term
#     mu : Float
#         The mixed effects coefficients
#     tau : Float
#         The hyperparameter of the variance of the random efffect

#     """
#     def __init__(self, X, y):        
#         # Initialization
#         self.p = X[0].shape[1]      # Number of variables
#         self.n = len(y)             # Number of sites
#         if isinstance(X[0], pd.DataFrame):
#             self.var_name = X[0].columns
#             self.X = [np.array(data) for data in X]
#             self.y = [np.array(outcome).reshape(len(outcome),1) for outcome in y]
#         else:
#             var_name = []
#             for i in range(self.p):
#                 var_name += ['X' + str(i+1)]
#             self.var_name = var_name
#             self.X = X
#             self.y = y
#         self.beta = np.repeat(0.1, self.p).reshape(self.p, 1)
#         self.lam = np.nan
#         self.mu = np.repeat(0.01, self.n)
#         self.tau = np.repeat(1.0, self.n)
#         self.df = pd.DataFrame
#         self.score = np.nan
#         self.predict = np.nan
#         self.time = np.nan
        
#     def fit(self, lam_it=11, lam_step=1, mu_it=3, beta_it=100, tau_it=20, bt_it=5):
#         # Iteration
#         pre_score = -10**10
#         for self.lam in np.arange(0, lam_it, lam_step):
#             start_time = time.time()
#             for step_mu in range(mu_it):
#                 for i in range(self.n):                    
#                     self.mu[i] = max_mu(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])
#                 print('mu:\n', self.mu, '\n')
#                 for step_theta in range(bt_it):
#                     theta_b = self.beta
#                     theta_t = self.tau
#                     for step in range(beta_it):
#                         lb1 = 0
#                         lb2 = 0
#                         for i in range(self.n):
#                             lb1 += lb_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])
#                             lb2 += lb_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])
#                         lb1 -= (2 * self.lam * self.beta.transpose())[0]
#                         lb2 -= np.diag(np.repeat(2 * self.lam, self.p))
#                         delta = lb1 @ inv(lb2)
#                         new_beta = self.beta - delta.reshape(self.p, 1)
#                         if max(np.abs(delta)) < 10 **(-6):
#     #                         print('Done with beta iteration')
#                             break;
#                         if max(np.abs(delta)) > 10 **(3):
#     #                         print('Get out of beta iteration (delta > 10^(-3)')
#                             break;
#                         if True in np.isnan(self.beta):
#     #                         print('Error: NaN beta')
#                             break;
#                         self.beta = new_beta
# #                     print('Step ', step + 1, ':\n')
#                     print('Beta:\n', self.beta, '\n')
# #                     print('Diff:\n', delta, '\n')
#                     print('Lam:\n', self.lam, '\n')
# #                     print('Score:\n',score,'\n')
#                     for step in range(tau_it):
#                         for i in range(self.n):
#                             lt1 = lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])
#                             lt2 = lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])
# #                             print('lt1', lt1, '\n')
# #                             print('lt2', lt2, '\n')
#                             delta_tau = lt1/lt2 #- 0.00000001
#                             print('delta:', delta_tau, '\n')
#                             new_tau = self.tau[i] - delta_tau
                            
#                             if np.abs(delta_tau) < 10 **(-5):
#                                 self.tau[i] = new_tau
#                                 break;
#                             if np.abs(delta_tau) > 10 **(2):
# #                                 self.tau = np.repeat(1.0, self.n)
#                                 break;
#                             if True in np.isnan(new_tau):
# #                                 self.tau = np.repeat(1.0, self.n)
#                                 break;
#                             if np.abs(new_tau) < 10**(-5):
# #                                 self.tau = np.repeat(1.0, self.n)
#                                 break;
#                             if new_tau < 0:
#                                 break;
#                             self.tau[i] = new_tau
#                     print('tau:', self.tau)
# #                     if (min(np.abs(self.beta - theta_b)) < 10 **(-3)) and (min(np.abs(self.tau - theta_t)) < 10 **(-3)):
# #                         break;
                   
#                 score = 0
#                 predict = []
#                 for i in range(len(self.X)):
#                     score += l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i]) - sum(self.lam * (self.beta) **2)
#                     predict += [Pi(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])]
#                 print('Score:\n',score,'\n')
#                 if True in np.isnan(score):
#                     break;
#                 if score > pre_score:
#                     optimized_beta = self.beta
#                     optimized_mu = self.mu
#                     optimized_lam = self.lam
#                     optimized_tau = self.tau
#                     # reset
#                     pre_score = score
#                     optimized_score = score
# #                 self.tau = np.repeat(1.0, self.n)
                
#         # Returning data
#         self.beta = optimized_beta
#         self.mu = optimized_mu
#         self.lam = optimized_lam
#         self.score = optimized_score
#         self.predict = np.concatenate(predict)      

#         X = np.concatenate(self.X)
        
#         y = np.concatenate(self.y)

#         V = np.diagflat(self.predict * (1 - self.predict))

#         SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(self.p,1)

#         Z = self.beta/SE

#         P = 2 * norm.cdf(-1 * np.abs(Z))

#         CI_025  = self.beta - 1.959964 * SE
#         CI_975  = self.beta + 1.959964 * SE

#         self.df = pd.DataFrame({'Coef': np.transpose(self.beta)[0], 'Std.Err': np.transpose(SE)[0],
#                            'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
#                            '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
#                           index = self.var_name)
        
#         return self

In [57]:
class LA:
    """
    A calss to run distributed GLMM

    ...

    Attributes
    ------------
    X : A list of DataFrames
        The data from different sites
    y : A list of arrays
        The binary outcomes from different sites
    beta : An array of params
        Fixed effects coefficents
    lam : Float
        Regularization term
    mu : Float
        The mixed effects coefficients
    tau : Float
        The hyperparameter of the variance of the random efffect

    """
    def __init__(self, X, y):        
        # Initialization
        self.p = X[0].shape[1]      # Number of variables
        self.n = len(y)             # Number of sites
        if isinstance(X[0], pd.DataFrame):
            self.var_name = X[0].columns
            self.X = [np.array(data) for data in X]
            self.y = [np.array(outcome).reshape(len(outcome),1) for outcome in y]
        else:
            var_name = []
            for i in range(self.p):
                var_name += ['X' + str(i+1)]
            self.var_name = var_name
            self.X = X
            self.y = y
        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
        self.lam = 0
        self.mu = np.repeat(0.5, self.n)
        self.tau = 0.5
#         self.tau = np.repeat(1.0, self.n)
#         self.tau = 0.62909983
        self.df = pd.DataFrame
        self.score = np.nan
        self.predict = np.nan
        self.time = np.nan
        
    def fit(self, lam_it=0, lam_step=1, mu_it=2, theta_it=100):
        # Iteration
        pre_score = -10**10
        for self.lam in np.arange(0, lam_it+lam_step, lam_step):
            converge = True
            print(f'In lambad = {self.lam}')
            for step_mu in range(mu_it):
                print(f'The {step_mu+1} step of mu')
                self.beta = self.beta.reshape(self.p, 1)
                for i in range(self.n):
#                     print('mu:\n', self.mu, '\n')
                    self.mu[i] = max_mu(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
#                 print('mu:\n', self.mu, '\n')
                for step_theta in range(theta_it):
                    theta = np.append(self.beta, self.tau)
#                     print('theta:\n', theta, '\n')
                    lb1 = 0
                    lb2 = 0
                    lt1 = 0
                    lt2 = 0
                    lbt2 = 0
#                     lt1 = []
#                     lt2 = []
                    for i in range(self.n):
                        lb1 += lb_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lb2 += lb_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lt1 += lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lt2 += lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lbt2 += lbt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
#                         lt1 = np.append(lt1, [lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])])
#                         lt2 = np.append(lt2, [lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])])
#                     print('diff of tau:\n',lt1/lt2,'\n')
                    lb1 -= (2 * self.lam * self.beta.transpose())[0]
                    lt1 -= 2 * self.lam * self.tau
                    lb2 -= np.diag(np.repeat(2 * self.lam, self.p))
                    lt2 = np.diag(lt2 - 2 * self.lam)#np.diag(lt2)
                    lt2 -= 0.00001
                    L1 = np.append(lb1, lt1)
#                     L2 = block_diag(lb2, lt2)
                    L2 = np.block([
                        [lb2, lbt2.reshape(self.p,1)],
                        [lbt2.reshape(1,self.p), lt2]
                    ])
                    delta = L1 @ inv(L2)
                    new_theta = theta - delta#.reshape(self.p+self.n, 1)
                    if (max(np.abs(delta[:-1])) < 10 **(-2)) and (delta[-1]<10**(-2)):
                        self.beta = new_theta[:self.p]
                        self.tau = new_theta[self.p:]
                        print('Done with iteration')
                        break;
                    if max(np.abs(delta)) > 10 **(2):
                        print('Get out of iteration (delta > 10^(2)')
                        break;
                    if True in np.isnan(new_theta[:self.p]):
                        print('Error: NaN beta, rested to 0')
                        # Reset beta and tau
                        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
                        break;
                    if True in np.isnan(new_theta[self.p:]):
                        print('Error: NaN tau, rested to 1.0')
                        # Reset beta and tau
                        self.tau = 1.0
                        break;
                    if True in np.isnan(self.mu):
                        print('Error: NaN mu, rested to 0')
                        # Reset beta and tau
                        self.mu = np.repeat(0.1, self.n)
                        break;                    
#                     if new_theta[-1] < 0:
# #                         print('Error: negative tau detectived')
#                         break;
                    self.beta = new_theta[:self.p].reshape(self.p, 1)
                    self.tau = new_theta[self.p:]
                    if step_theta == theta_it-1:
                        print('Error: Did not converged, reset all\n')
                        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
                        self.mu = np.repeat(0.1, self.n)
                        self.tau = 1.0
                        converge = False
                        break;
#                     print('Step ', step_theta + 1, ':\n')
#                     print('delta:', delta, '\n')
#                     print('Beta:\n', self.beta, '\n')
#                     print('Diff:\n', delta, '\n')
#                     print('Lam:\n', self.lam, '\n')
#                     print('Score:\n',score,'\n')
#                     print('tau:', self.tau, '\n')
#                     print('mu:\n', self.mu, '\n')
                if not converge:
                    break;
#                 else:
#                     continue
#                 break;
                    
#                     print('l:\n',l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau) - sum(self.lam * (self.beta) **2),'\n')

                score = 0
                predict = []
                self.beta = new_theta[:self.p].reshape(self.p, 1)
                for i in range(self.n):
    #                     print('Lam:\n', self.lam, '\n')
    #                     print('tau:', self.tau, '\n')
    #                     print('Beta:\n', self.beta, '\n')
                    score += l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau) - sum(self.lam * (self.beta) **2)
                    predict += [Pi(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)]
                print('score:\n', score, '\n')
                if True in np.isnan(score):
                    print('Error: NaN score')
                    break;
                if (np.mean(score) > pre_score) and converge:
                    optimized_beta = self.beta
                    optimized_mu = self.mu
                    optimized_lam = self.lam
                    optimized_tau = self.tau
                    # reset
                    pre_score = score
                    optimized_score = score
    #                 self.tau = np.repeat(1.0, self.n)

        # Returning data
        self.beta = optimized_beta
        self.mu = optimized_mu
        self.lam = optimized_lam
        self.tau = optimized_tau
        self.score = optimized_score
        self.predict = np.concatenate(predict)      

        X = np.concatenate(self.X)
        
        y = np.concatenate(self.y)

        V = np.diagflat(self.predict * (1 - self.predict) + 0.0001)

        SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(self.p,1)

        Z = self.beta/SE

        P = 2 * norm.cdf(-1 * np.abs(Z))

        CI_025  = self.beta - 1.959964 * SE
        CI_975  = self.beta + 1.959964 * SE

        self.df = pd.DataFrame({'Coef': np.transpose(self.beta)[0], 'Std.Err': np.transpose(SE)[0],
                           'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                           '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                          index = self.var_name)
        
        return self

In [98]:
truth = np.array([-1.5,0.1,-0.5,-0.3,0.4,-0.2,-0.25,0.35,-0.1,0.5]).reshape(10, 1)
var_name = []
for i in range(10):
    var_name += ['X' + str(i+1)]
df = pd.read_csv('/Users/wli17/Documents/GLMM/Simulation_data_GLMM/Setting_6/2Sites_30PatientsEachSite_LargeVar4_Dataset18.csv', index_col=0)
data1 = df[df['Site_ID'] == 1][var_name]
data2 = df[df['Site_ID'] == 2][var_name]
out1 = np.array(df[df['Site_ID'] == 1]['y']).reshape(30,1)
out2 = np.array(df[df['Site_ID'] == 2]['y']).reshape(30,1)
data = [data1, data2]
out = [out1, out2]

In [99]:
truth = np.array([-1.5,0.1,-0.5,-0.3,0.4,-0.2,-0.25,0.35,-0.1,0.5]).reshape(10, 1)
var_name = []
for i in range(10):
    var_name += ['X' + str(i+1)]
df = pd.read_csv('/Users/wli17/Documents/GLMM/Simulation_data_GLMM/Setting_7/10Sites_30PatientsEachSite_SmallVar1_Dataset18.csv', index_col=0)
data = []
for k in range(10):
    globals()['data%s' % str(k+1)] = np.array(df[df['Site_ID'] == k+1][var_name])
    data.append(globals()['data%s' % str(k+1)])
out = []
for k in range(10):
    globals()['out%s' % str(k+1)] = np.array(df[df['Site_ID'] == k+1]['y']).reshape(30,1)
    out.append(globals()['out%s' % str(k+1)])

In [91]:
# def Add_intercept(data):
#     for x in data:
#         x.insert(loc=0, column='Intercept', value=1)
#     return data

In [92]:
# X = Add_intercept([pd.DataFrame(X1), pd.DataFrame(X2)])

In [69]:
class LA:
    """
    A calss to run distributed GLMM

    ...

    Attributes
    ------------
    X : A list of DataFrames
        The data from different sites
    y : A list of arrays
        The binary outcomes from different sites
    beta : An array of params
        Fixed effects coefficents
    lam : Float
        Regularization term
    mu : Float
        The mixed effects coefficients
    tau : Float
        The hyperparameter of the variance of the random efffect

    """
    def __init__(self, X, y):        
        # Initialization
        self.p = X[0].shape[1]      # Number of variables
        self.n = len(y)             # Number of sites
        if isinstance(X[0], pd.DataFrame):
            self.var_name = X[0].columns
            self.X = [np.array(data) for data in X]
            self.y = [np.array(outcome).reshape(len(outcome),1) for outcome in y]
        else:
            var_name = []
            for i in range(self.p):
                var_name += ['X' + str(i+1)]
            self.var_name = var_name
            self.X = X
            self.y = y
        self.beta = np.repeat(0.1, self.p).reshape(self.p, 1)
        self.lam = 0
        self.mu = np.repeat(1, self.n)
        self.tau = 1
#         self.tau = np.repeat(1.0, self.n)
#         self.tau = 0.62909983
        self.df = pd.DataFrame
        self.score = np.nan
        self.predict = np.nan
        self.time = np.nan
        
    def fit(self, lam_it=0, lam_step=1, mu_it=2, theta_it=100):
        # Iteration
        pre_score = -10**10
        for self.lam in np.arange(0, lam_it+lam_step, lam_step):
            print(f'In lambad = {self.lam}')
            for step_mu in range(mu_it):
                print(f'The {step_mu+1} step of mu')
                self.beta = self.beta.reshape(self.p, 1)
                for i in range(self.n):
#                     print('mu:\n', self.mu, '\n')
                    self.mu[i] = max_mu(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
#                 print('mu:\n', self.mu, '\n')
                for step_theta in range(theta_it):
                    theta = np.append(self.beta, self.tau)
#                     print('theta:\n', theta, '\n')
                    lb1 = 0
                    lb2 = 0
                    lt1 = 0
                    lt2 = 0
                    lbt2 = 0
#                     lt1 = []
#                     lt2 = []
                    for i in range(self.n):
                        lb1 += lb_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lb2 += lb_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lt1 += lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lt2 += lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lbt2 += lbt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
#                         lt1 = np.append(lt1, [lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])])
#                         lt2 = np.append(lt2, [lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])])
#                     print('diff of tau:\n',lt1/lt2,'\n')
                    lb1 -= (2 * self.lam * self.beta.transpose())[0]
                    lt1 -= 2 * self.lam * self.tau
                    lb2 -= np.diag(np.repeat(2 * self.lam, self.p)) + 0.00001
                    lt2 = np.diag(lt2 - 2 * self.lam)#np.diag(lt2)
                    lt2 -= 0.00001
                    L1 = np.append(lb1, lt1)
#                     L2 = block_diag(lb2, lt2)
                    L2 = np.block([
                        [lb2, lbt2.reshape(self.p,1)],
                        [lbt2.reshape(1,self.p), lt2]
                    ])
                    delta = L1 @ inv(L2)
                    new_theta = theta - delta#.reshape(self.p+self.n, 1)
                    if (max(np.abs(delta[:-1])) < 10 **(-2)) and (delta[-1]<10**(-2)):
                        self.beta = new_theta[:self.p]
                        self.tau = new_theta[self.p:]
                        converge = True
                        print('Done with iteration')
                        break;
                    if max(np.abs(delta)) > 10 **(2):
                        converge = False
                        print('Get out of iteration (delta > 10^(2)')
                        break;
                    if True in np.isnan(new_theta[:self.p]):
                        print('Error: NaN beta, rested to 0')
                        # Reset beta and tau
                        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
                        break;
                    if True in np.isnan(new_theta[self.p:]):
                        print('Error: NaN tau, rested to 1.0')
                        # Reset beta and tau
                        converge = False
                        self.tau = 1.0
                        break;
                    if True in np.isnan(self.mu):
                        print('Error: NaN mu, rested to 0')
                        # Reset beta and tau
                        converge = False
                        self.mu = np.repeat(0.1, self.n)
                        break;                    
#                     if new_theta[-1] < 0:
# #                         print('Error: negative tau detectived')
#                         break;
                    self.beta = new_theta[:self.p].reshape(self.p, 1)
                    self.tau = new_theta[self.p:]
                    if step_theta == theta_it-1:
                        print('Error: Did not converged, reset all\n')
                        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
                        self.mu = np.repeat(1, self.n)
                        self.tau = 1
                        converge = False
                        break;
#                     print('Step ', step_theta + 1, ':\n')
#                     print('delta:', delta, '\n')
#                     print('Beta:\n', self.beta, '\n')
#                     print('Diff:\n', delta, '\n')
#                     print('Lam:\n', self.lam, '\n')
#                     print('Score:\n',score,'\n')
#                     print('tau:', self.tau, '\n')
#                     print('mu:\n', self.mu, '\n')
#                 if not converge:
#                     break;
#                 else:
#                     continue
#                 break;
                    
#                     print('l:\n',l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau) - sum(self.lam * (self.beta) **2),'\n')

            score = 0
            predict = []
            self.beta = new_theta[:self.p].reshape(self.p, 1)
            for i in range(self.n):
#                     print('Lam:\n', self.lam, '\n')
#                     print('tau:', self.tau, '\n')
#                     print('Beta:\n', self.beta, '\n')
                score += l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau) - sum(self.lam * (self.beta) **2)
                predict += [Pi(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)]
            print('score:\n', score, '\n')
            if True in np.isnan(score):
                print('Error: NaN score')
                break;
            if (np.mean(score) > pre_score) and converge:
                optimized_beta = self.beta
                optimized_mu = self.mu
                optimized_lam = self.lam
                optimized_tau = self.tau
                # reset
                pre_score = score
                optimized_score = score
#                 self.tau = np.repeat(1.0, self.n)

        # Returning data
        optimized_beta[0] = optimized_beta[0] + np.mean(optimized_tau * optimized_mu)
        self.beta = optimized_beta
        self.mu = optimized_mu
        self.lam = optimized_lam
        self.tau = optimized_tau
        self.score = optimized_score
        self.predict = np.concatenate(predict)


        X = np.concatenate(self.X)
        
        y = np.concatenate(self.y)

        V = np.diagflat(self.predict * (1 - self.predict) + 0.0001)

        SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(self.p,1)

        Z = self.beta/SE

        P = 2 * norm.cdf(-1 * np.abs(Z))

        CI_025  = self.beta - 1.959964 * SE
        CI_975  = self.beta + 1.959964 * SE

        self.df = pd.DataFrame({'Coef': np.transpose(self.beta)[0], 'Std.Err': np.transpose(SE)[0],
                           'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                           '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                          index = self.var_name)
        
        return self

In [117]:
model=0
model = LA(data, out).fit(lam_it=1)

In lambad = 0
The 1 step of mu
Done with iteration
The 2 step of mu
Error: Did not converged, reset all

score:
 [4.9887165] 

In lambad = 1
The 1 step of mu
Get out of iteration (delta > 10^(2)
The 2 step of mu
Get out of iteration (delta > 10^(2)
score:
 [-116879.03156202] 



UnboundLocalError: local variable 'optimized_beta' referenced before assignment

In [ ]:
model.df

In [ ]:
model.tau

In [ ]:
model.mu

In [ ]:
(model.tau * model.mu + model.df.Coef[0]).mean()

In [ ]:
model.lam

In [ ]:
model.score

In [904]:
2 * norm.cdf(-1 * np.abs(0))

1.0

In [73]:
p = 10
np.random.seed(233)
true_beta = (np.random.rand(p,1) - np.random.rand(p,1)) * 10
true_tau = np.random.rand(1)
true_mu = np.random.rand(2)
X1 = (np.random.rand(1000, p) - np.random.rand(1000, p)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(true_mu[0], true_tau, 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, p) - np.random.rand(1000, p)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(true_mu[1], true_tau, 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)
X = [X1, X2]
y = [y1, y2]

In [700]:
pd.DataFrame(X1).to_csv('/Users/wli17/Documents/GLMM/GLMM/X1_test.csv', header=None, index=None)
pd.DataFrame(X2).to_csv('/Users/wli17/Documents/GLMM/GLMM/X2_test.csv', header=None, index=None)
pd.DataFrame(y1).to_csv('/Users/wli17/Documents/GLMM/GLMM/y1_test.csv', header=None, index=None)
pd.DataFrame(y2).to_csv('/Users/wli17/Documents/GLMM/GLMM/y2_test.csv', header=None, index=None)

In [74]:
true_tau

array([0.42244813])

In [75]:
true_mu

array([0.2513114 , 0.53939061])

In [76]:
model = LA(X, y).fit(lam_it=5)

In lambad = 0
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [0.42859166] 

In lambad = 1
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-48.19082457] 

In lambad = 2
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-65.01836753] 

In lambad = 3
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-76.48262169] 

In lambad = 4
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-85.42383709] 

In lambad = 5
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-92.87641947] 



In [77]:
model.score

array([0.42859166])

In [78]:
model.df

,Coef,Std.Err,z,P-value,[0.025,0.975]
X1,-1.852312,0.046169,-40.120639,0.000000e+00,-1.942801,-1.761823
X2,4.455123,0.090925,48.997532,0.000000e+00,4.276913,4.633334
X3,-2.748092,0.061414,-44.746988,0.000000e+00,-2.868461,-2.627722
X4,-0.388153,0.030710,-12.639486,1.278871e-36,-0.448342,-0.327963
X5,2.487525,0.056635,43.922352,0.000000e+00,2.376523,2.598526
X6,-5.265764,0.106590,-49.402211,0.000000e+00,-5.474675,-5.056852
X7,-5.922779,0.118307,-50.062845,0.000000e+00,-6.154657,-5.690902
X8,-1.364450,0.039588,-34.466528,2.546482e-260,-1.442041,-1.286860
X9,-1.226655,0.037193,-32.981129,1.514630e-238,-1.299551,-1.153759
X10,-0.280893,0.029681,-9.463607,2.975004e-21,-0.339067,-0.222718


In [79]:
model.lam

0

In [80]:
model.tau

array([28.67927709])

In [81]:
model.mu

array([0, 0])

In [82]:
true_beta

array([[-1.97611797],
       [ 5.10199293],
       [-3.01214046],
       [-0.3198761 ],
       [ 2.95956433],
       [-5.80965677],
       [-6.57774089],
       [-1.42889042],
       [-1.39586908],
       [-0.24159223]])

In [83]:
model.beta

array([[-1.85231217],
       [ 4.45512346],
       [-2.74809173],
       [-0.38815264],
       [ 2.48752452],
       [-5.26576353],
       [-5.92277945],
       [-1.36445049],
       [-1.22665506],
       [-0.28089259]])

In [575]:
true_tau

array([0.85762553])

In [ ]:
model.score

In [ ]:
np.concatenate((lb_1(LA(X, y).X[1], LA(X, y).y[1], LA(X, y).mu[1], LA(X, y).beta, LA(X, y).tau[1]),
np.array(lt1)))

In [283]:
lbt_2(LA(X, y).X[0], LA(X, y).y[0], LA(X, y).mu[0], LA(X, y).beta, LA(X, y).tau).reshape(10,1)

array([[2.49327659],
       [4.40395284],
       [3.30618879],
       [3.12731818],
       [3.63733157],
       [3.44444707],
       [3.42738627],
       [3.47021423],
       [3.84349734],
       [3.66983315]])

In [ ]:
lt2 = []

In [ ]:
lt2 = np.append(lt2, [lt_2(LA(X, y).X[1], LA(X, y).y[1], LA(X, y).mu[1], LA(X, y).beta, LA(X, y).tau[1])])

In [ ]:
lt_1(LA(X, y).X[1], LA(X, y).y[1], LA(X, y).mu[1], LA(X, y).beta, LA(X, y).tau[1])/lt_2(LA(X, y).X[1], LA(X, y).y[1], LA(X, y).mu[1], LA(X, y).beta, LA(X, y).tau[1])

In [ ]:
np.diag(lt2 - 2*LA(X, y).tau)

In [ ]:
lt2 - LA(X, y).tau

In [280]:
np.eye(3) * 3

array([[3., 0., 0.],
       [0., 3., 0.],
       [0., 0., 3.]])

## Convergence rate

In [85]:
from numpy.linalg import inv
import pandas as pd
import numpy as np
import time
from scipy.stats import norm
from scipy.linalg import block_diag
def logsumexp(x):
    c = max(x)
    LSE = c + np.log(np.sum(np.exp(x - c)))
    return np.exp(x - LSE)
def Pi(x, y, mu, beta, tau):
    result = np.asarray((np.exp(x @ beta + tau * mu) / (1 + np.exp(x @ beta + tau * mu))))
    # Check if there are exp(0) cases, if true, return \pi = 1 correspondingly
    return np.nan_to_num(result, nan = 1)
def g(x, y, mu, beta, tau):
    g = sum(y * logsumexp(Pi(x, y, mu, beta, tau)) + (1 - y) * logsumexp(1 - Pi(x, y, mu, beta, tau))) \
    + np.nan_to_num(np.log(tau * (np.sqrt(2 * np.pi))**(-1) * np.exp(-mu**2/2)), nan=0)
    return g
def g_u(x, y, mu, beta, tau):
    return tau * sum((y - Pi(x, y, mu, beta, tau))) - mu
def g_b(x, y, mu, beta, tau):
    return np.sum(x * y - x * Pi(x, y, mu, beta, tau), axis = 0)
def g_t(x, y, mu, beta, tau):
    return mu * sum((y - Pi(x, y, mu, beta, tau)))
def g_uu(x, y, mu, beta, tau):
    result = np.nan_to_num(tau * np.asarray((np.exp(x @ beta + tau * mu) /\
                                              (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return - tau * sum(result) - 1
def g_bb(x, y, mu, beta, tau):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
        * np.nan_to_num((np.exp(x[i] @ beta + tau * mu) / (1 + np.exp(x[i] @ beta + tau * tau * mu))**2), nan = 0))
    return result
def g_tt(x, y, mu, beta, tau):
    result = np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                              (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return -mu * sum(result)
def g_ub(x, y, mu, beta, tau):
    result = np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) /\
                                       (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return -tau * np.sum(x * result, axis = 0)
def g_ut(x, y, mu, beta, tau):
    result = sum(y - Pi(x, y, mu, beta, tau))\
    - tau * sum(mu * np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                          (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0))
    return result
def g_bt(x, y, mu, beta, tau):
    result = np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                          (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    return -np.sum(x * result, axis = 0)
def g_uuu(x, y, mu, beta, tau):
    return -tau * sum(-tau**2 * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0))
def g_uub(x, y, mu, beta, tau): # add -
    return -tau * np.sum(-tau * x * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0), axis = 0)
def g_ubb(x, y, mu, beta, tau):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                         * np.nan_to_num((np.exp(x[i] @ beta + tau * mu) * (np.exp(x[i] @ beta + tau * mu) - 1)\
                                       / (1 + np.exp(x[i] @ beta + tau * mu))**3), nan = 0))
    return -tau * result
def g_uut(x, y, mu, beta, tau):
    result = -tau * np.sum(np.nan_to_num((np.exp(x @ beta + tau * mu) * ((1 - tau * mu) * np.exp(x @ beta + tau * mu) + tau * mu + 1))\
    / (1 + np.exp(x @ beta + tau * mu))**3, nan = 0), axis = 0) - np.sum(np.nan_to_num(tau * np.asarray((np.exp(x @ beta + tau * mu) /\
                                              (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0))
    return result
def g_utt(x, y, mu, beta, tau):
    result = -2 * np.sum(np.nan_to_num(mu * np.asarray((np.exp(x @ beta + tau * mu) /\
                                          (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0), axis = 0)\
    - tau * np.sum( np.nan_to_num(-mu**2 * np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1) /\
                   (1 + np.exp(x @ beta + tau * mu))**3, nan = 0) )
    return result
def g_ubt(x, y, mu, beta, tau):
    left = np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) /\
                                       (1 + np.exp(x @ beta + tau * mu))**2)), nan = 0)
    left = -np.sum(x * left, axis = 0)
    right = np.nan_to_num(np.asarray((mu * np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1))/\
                                    (np.exp(x @ beta + tau * mu) + 1)**3), nan = 0)
    right = -np.sum(x * right, axis = 0)
    return left + right
def g_uuuu(x, y, mu, beta, tau):
    result = -tau * np.sum( np.nan_to_num(-tau**3 * np.exp(2 * x @ beta + 2 * tau * mu)/(np.exp(x @ beta + tau * mu) + 1)**3\
                           - tau**3 * np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)/(np.exp(x @ beta + tau * mu) + 1)**3\
                           + 3 * tau**3 * np.exp(2 * x @ beta + 2 * tau * mu) * (np.exp(x @ beta + tau * mu) - 1)/(np.exp(x @ beta + tau * mu) + 1)**4, nan = 0) )
    return result
def g_uuub(x, y, mu, beta, tau):
    result = -tau * np.sum( np.nan_to_num(tau**2 * x * np.exp(x @ beta + tau * mu)\
                           * (-4 * np.exp(x @ beta + tau * mu) + np.exp(2 * x @ beta + 2 * tau * mu) + 1)\
                           /(np.exp(x @ beta + tau * mu) + 1)**4, nan = 0), axis = 0 )
    return result
def g_uubb(x, y, mu, beta, tau):
    result = 0
    for i in range(len(y)):
        result += x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1]) * np.nan_to_num(np.exp(x[i] @ beta + tau * mu)\
        * (-4 * np.exp(x[i] @ beta + tau * mu) + np.exp(2 * x[i] @ beta + 2 * tau * mu) + 1)\
        / (np.exp(x[i] @ beta + tau * mu) + 1)**4, nan=0)
        
    result *= tau                     
        
    return -tau * result
def g_uuut(x, y, mu, beta, tau):
    left = -np.sum(-tau**2 * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0))
    right = -tau * tau * np.sum(np.nan_to_num(np.exp(x @ beta + tau * mu) * (-4 * tau * mu * np.exp(x @ beta + tau * mu)\
                                                        + tau * mu * np.exp(2 * x @ beta + 2 * tau * mu)\
                                                        -2 * np.exp(2 * x @ beta + 2 * tau * mu) + tau * mu + 2)\
           /(np.exp(x @ beta + tau * mu) + 1)**4, nan=0))
    return left + right
def g_uutt(x, y, mu, beta, tau):
    left = -2 * np.sum(np.nan_to_num((np.exp(x @ beta + tau * mu)\
                                  * ((1 - tau * mu) * np.exp(x @ beta + tau * mu) + tau * mu + 1))\
    / (1 + np.exp(x @ beta + tau * mu))**3, nan = 0), axis = 0)
    right = -tau * mu * np.sum(np.nan_to_num(np.exp(x @ beta + tau * mu) * (-4 * tau * mu * np.exp(x @ beta + tau * mu)\
                                                        + tau * mu * np.exp(2 * x @ beta + 2 * tau * mu)\
                                                        -2 * np.exp(2 * x @ beta + 2 * tau * mu) + tau * mu + 2)\
           /(np.exp(x @ beta + tau * mu) + 1)**4, nan=0))
    return left + right
def g_uubt(x, y, mu, beta, tau):
    left = -np.sum(-tau * x * np.nan_to_num(np.asarray((np.exp(x @ beta + tau * mu) * (np.exp(x @ beta + tau * mu) - 1)\
                             / (1 + np.exp(x @ beta + tau * mu))**3)), nan = 0), axis = 0)
    right = -tau * np.sum(x * np.nan_to_num(np.asarray(np.exp(x @ beta + tau * mu) * (-4 * tau * mu * np.exp(x @ beta + tau * mu)\
                                                             + tau * mu * np.exp(2 * x @ beta + 2 * tau * mu)\
                                                             - np.exp(2 * x @ beta + 2 * tau * mu) + tau * mu + 1)\
                          / (np.exp(x @ beta + tau * mu) + 1)**4), nan = 0), axis = 0)
    return left + right
def omega(x, y, mu, beta, tau):
    return np.sqrt(-1/g_uu(x, y, mu, beta, tau))
def mu_b(x, y, mu, beta, tau):
    return omega(x, y, mu, beta, tau)**2 * g_ub(x, y, mu, beta, tau)
def mu_bb(x, y, mu, beta, tau):
    result = omega(x, y, mu, beta, tau)**2 * (mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
                                              @ mu_b(x, y, mu, beta, tau).reshape(1,x.shape[1])\
                                              * g_uuu(x, y, mu, beta, tau)\
                                              + 2 * mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
                                              @ g_uub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
                                              + g_ubb(x, y, mu, beta, tau))
    return result
def omega_b(x, y, mu, beta, tau):
    return 0.5 * omega(x, y, mu, beta, tau)**3 * (g_uuu(x, y, mu, beta, tau) * mu_b(x, y, mu, beta, tau)\
                                                    + g_uub(x, y, mu, beta, tau))
def mu_t(x, y, mu, beta, tau):
    return omega(x, y, mu, beta, tau)**2 * g_ut(x, y, mu, beta, tau)
def mu_tt(x, y, mu, beta, tau):
    result = omega(x, y, mu, beta, tau)**2 * (mu_t(x, y, mu, beta, tau)\
                                              * mu_t(x, y, mu, beta, tau)\
                                              * g_uuu(x, y, mu, beta, tau)\
                                              + 2 * mu_t(x, y, mu, beta, tau)\
                                              * g_uut(x, y, mu, beta, tau)\
                                              + g_utt(x, y, mu, beta, tau))
    return result
def mu_bt(x, y, mu, beta, tau):
    result = omega(x, y, mu, beta, tau)**2 * (mu_b(x, y, mu, beta, tau)\
                                              * mu_t(x, y, mu, beta, tau)\
                                              * g_uuu(x, y, mu, beta, tau)\
                                              + mu_b(x, y, mu, beta, tau)\
                                              * g_uut(x, y, mu, beta, tau)\
                                              + mu_t(x, y, mu, beta, tau)\
                                              * g_uub(x, y, mu, beta, tau)\
                                              + g_ubt(x, y, mu, beta, tau))
    return result
def omega_t(x, y, mu, beta, tau):
    return 0.5 * omega(x, y, mu, beta, tau)**3 * (g_uuu(x, y, mu, beta, tau) * mu_t(x, y, mu, beta, tau)\
                                                    + g_uut(x, y, mu, beta, tau))
def omega_bb(x, y, mu, beta, tau):
    result = 3/4 * omega(x, y, mu, beta, tau)**5 * (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
                                                      + g_uub(x, y, mu, beta, tau)).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + g_uub(x, y, mu, beta, tau)).reshape(1,x.shape[1]) + 1/2 * omega(x, y, mu, beta, tau)**3\
     * (mu_bb(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + (mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
           @ mu_b(x, y, mu, beta, tau).reshape(1,x.shape[1]) * g_uuuu(x, y, mu, beta, tau))\
        + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
        + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
        + g_uubb(x, y, mu, beta, tau)
       )
    return result
def omega_tt(x, y, mu, beta, tau):
    result = 3/4 * omega(x, y, mu, beta, tau)**5 * (mu_t(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
                                                      + g_uut(x, y, mu, beta, tau))\
     * (mu_t(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + g_uut(x, y, mu, beta, tau)) + 1/2 * omega(x, y, mu, beta, tau)**3\
     * (mu_tt(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + (mu_t(x, y, mu, beta, tau)\
           * mu_t(x, y, mu, beta, tau) * g_uuuu(x, y, mu, beta, tau))\
        + mu_t(x, y, mu, beta, tau) * g_uuut(x, y, mu, beta, tau)\
        + mu_t(x, y, mu, beta, tau) * g_uuut(x, y, mu, beta, tau)\
        + g_uutt(x, y, mu, beta, tau)
       )
    return result
def omega_bt(x, y, mu, beta, tau):
    result = 3/4 * omega(x, y, mu, beta, tau)**5 * (mu_t(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
                                                      + g_uut(x, y, mu, beta, tau))\
     * (mu_b(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + g_uub(x, y, mu, beta, tau)) + 1/2 * omega(x, y, mu, beta, tau)**3\
     * (mu_bt(x, y, mu, beta, tau) * g_uuu(x, y, mu, beta, tau)\
        + (mu_b(x, y, mu, beta, tau)\
           * mu_t(x, y, mu, beta, tau) * g_uuuu(x, y, mu, beta, tau))\
        + mu_b(x, y, mu, beta, tau) * g_uuut(x, y, mu, beta, tau)\
        + mu_t(x, y, mu, beta, tau) * g_uuub(x, y, mu, beta, tau)\
        + g_uubt(x, y, mu, beta, tau)
       )
    return result
def lb_1(x, y, mu, beta, tau):
    l1 = omega_b(x, y, mu, beta, tau) / omega(x, y, mu, beta, tau)\
    + g_u(x, y, mu, beta, tau) * mu_b(x, y, mu, beta, tau) + g_b(x, y, mu, beta, tau)
    return l1
def lt_1(x, y, mu, beta, tau):
    l1 = omega_t(x, y, mu, beta, tau) / omega(x, y, mu, beta, tau)\
    + g_u(x, y, mu, beta, tau) * mu_t(x, y, mu, beta, tau) + g_t(x, y, mu, beta, tau)
    return l1
def lb_2(x, y, mu, beta, tau):
    l2 = omega(x, y, mu, beta, tau)**(-2) * (omega_bb(x, y, mu, beta, tau) * omega(x, y, mu, beta, tau)\
                                               - omega_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
                                               @ omega_b(x, y, mu, beta, tau).reshape(1,x.shape[1]))\
     + mu_bb(x, y, mu, beta, tau) * g_u(x, y, mu, beta, tau) + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta, tau) * g_uu(x, y, mu, beta, tau) + g_ub(x, y, mu, beta, tau)).reshape(1,x.shape[1])\
     + mu_b(x, y, mu, beta, tau).reshape(x.shape[1],1) @ g_ub(x, y, mu, beta, tau).reshape(1,x.shape[1])\
     + g_bb(x, y, mu, beta, tau)
    return l2
def lt_2(x, y, mu, beta, tau):
    l2 = omega(x, y, mu, beta, tau)**(-2) * (omega_tt(x, y, mu, beta, tau) * omega(x, y, mu, beta, tau)\
                                               - omega_t(x, y, mu, beta, tau)\
                                               * omega_t(x, y, mu, beta, tau))\
     + mu_tt(x, y, mu, beta, tau) * g_u(x, y, mu, beta, tau) + mu_t(x, y, mu, beta, tau)\
     * (mu_t(x, y, mu, beta, tau) * g_uu(x, y, mu, beta, tau) + g_ut(x, y, mu, beta, tau))\
     + mu_t(x, y, mu, beta, tau) * g_ut(x, y, mu, beta, tau)\
     + g_tt(x, y, mu, beta, tau)
    return l2
def lbt_2(x, y, mu, beta, tau):
    l2 = omega(x, y, mu, beta, tau)**(-2) * (omega_bt(x, y, mu, beta, tau) * omega(x, y, mu, beta, tau)\
                                               - omega_b(x, y, mu, beta, tau)\
                                               * omega_t(x, y, mu, beta, tau))\
     + mu_bt(x, y, mu, beta, tau) * g_u(x, y, mu, beta, tau) + mu_b(x, y, mu, beta, tau)\
     * (mu_t(x, y, mu, beta, tau) * g_uu(x, y, mu, beta, tau) + g_ut(x, y, mu, beta, tau))\
     + mu_t(x, y, mu, beta, tau) * g_ub(x, y, mu, beta, tau)\
     + g_bt(x, y, mu, beta, tau)
    return l2
def max_mu(x, y, mu, beta, tau, max_iter=100):
    for step in range(max_iter):
        mu_new = mu - g_u(x, y, mu, beta, tau)/g_uu(x, y, mu, beta, tau)
        diff = mu_new - mu
        mu = mu_new
        if np.abs(diff) < 10**(-10):
            break;       
    return mu
def l(x, y, mu, beta, tau):
    l = 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta, tau)) + g(x, y, mu, beta, tau)
    return l
class LA:
    """
    A calss to run distributed GLMM

    ...

    Attributes
    ------------
    X : A list of DataFrames
        The data from different sites
    y : A list of arrays
        The binary outcomes from different sites
    beta : An array of params
        Fixed effects coefficents
    lam : Float
        Regularization term
    mu : Float
        The mixed effects coefficients
    tau : Float
        The hyperparameter of the variance of the random efffect

    """
    def __init__(self, X, y):        
        # Initialization
        self.p = X[0].shape[1]      # Number of variables
        self.n = len(y)             # Number of sites
        if isinstance(X[0], pd.DataFrame):
            self.var_name = X[0].columns
            self.X = [np.array(data) for data in X]
            self.y = [np.array(outcome).reshape(len(outcome),1) for outcome in y]
        else:
            var_name = []
            for i in range(self.p):
                var_name += ['X' + str(i+1)]
            self.var_name = var_name
            self.X = X
            self.y = y
        self.beta = np.repeat(0.1, self.p).reshape(self.p, 1)
        self.lam = 0
        self.mu = np.repeat(1, self.n)
        self.tau = 1
#         self.tau = np.repeat(1.0, self.n)
#         self.tau = 0.62909983
        self.df = pd.DataFrame
        self.score = np.nan
        self.predict = np.nan
        self.time = np.nan
        
    def fit(self, lam_it=0, lam_step=1, mu_it=2, theta_it=100):
        # Iteration
        pre_score = -10**10
        for self.lam in np.arange(0, lam_it+lam_step, lam_step):
            print(f'In lambad = {self.lam}')
            for step_mu in range(mu_it):
                print(f'The {step_mu+1} step of mu')
                self.beta = self.beta.reshape(self.p, 1)
                for i in range(self.n):
#                     print('mu:\n', self.mu, '\n')
                    self.mu[i] = max_mu(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
#                 print('mu:\n', self.mu, '\n')
                for step_theta in range(theta_it):
                    theta = np.append(self.beta, self.tau)
#                     print('theta:\n', theta, '\n')
                    lb1 = 0
                    lb2 = 0
                    lt1 = 0
                    lt2 = 0
                    lbt2 = 0
#                     lt1 = []
#                     lt2 = []
                    for i in range(self.n):
                        lb1 += lb_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lb2 += lb_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lt1 += lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lt2 += lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
                        lbt2 += lbt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)
#                         lt1 = np.append(lt1, [lt_1(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])])
#                         lt2 = np.append(lt2, [lt_2(self.X[i], self.y[i], self.mu[i], self.beta, self.tau[i])])
#                     print('diff of tau:\n',lt1/lt2,'\n')
                    lb1 -= (2 * self.lam * self.beta.transpose())[0]
                    lt1 -= 2 * self.lam * self.tau
                    lb2 -= np.diag(np.repeat(2 * self.lam, self.p)) + 0.00001
                    lt2 = np.diag(lt2 - 2 * self.lam)#np.diag(lt2)
                    lt2 -= 0.00001
                    L1 = np.append(lb1, lt1)
#                     L2 = block_diag(lb2, lt2)
                    L2 = np.block([
                        [lb2, lbt2.reshape(self.p,1)],
                        [lbt2.reshape(1,self.p), lt2]
                    ])
                    delta = L1 @ inv(L2)
                    new_theta = theta - delta#.reshape(self.p+self.n, 1)
                    if (max(np.abs(delta[:-1])) < 10 **(-2)) and (delta[-1]<10**(-2)):
                        self.beta = new_theta[:self.p]
                        self.tau = new_theta[self.p:]
                        converge = True
                        print('Done with iteration')
                        break;
                    if max(np.abs(delta)) > 10 **(2):
                        converge = False
                        print('Get out of iteration (delta > 10^(2)')
                        break;
                    if True in np.isnan(new_theta[:self.p]):
                        print('Error: NaN beta, rested to 0')
                        # Reset beta and tau
                        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
                        break;
                    if True in np.isnan(new_theta[self.p:]):
                        print('Error: NaN tau, rested to 1.0')
                        # Reset beta and tau
                        converge = False
                        self.tau = 1.0
                        break;
                    if True in np.isnan(self.mu):
                        print('Error: NaN mu, rested to 0')
                        # Reset beta and tau
                        converge = False
                        self.mu = np.repeat(0.1, self.n)
                        break;                    
#                     if new_theta[-1] < 0:
# #                         print('Error: negative tau detectived')
#                         break;
                    self.beta = new_theta[:self.p].reshape(self.p, 1)
                    self.tau = new_theta[self.p:]
                    if step_theta == theta_it-1:
                        print('Error: Did not converged, reset all\n')
                        self.beta = np.repeat(0, self.p).reshape(self.p, 1)
                        self.mu = np.repeat(1, self.n)
                        self.tau = 1
                        converge = False
                        break;
#                     print('Step ', step_theta + 1, ':\n')
#                     print('delta:', delta, '\n')
#                     print('Beta:\n', self.beta, '\n')
#                     print('Diff:\n', delta, '\n')
#                     print('Lam:\n', self.lam, '\n')
#                     print('Score:\n',score,'\n')
#                     print('tau:', self.tau, '\n')
#                     print('mu:\n', self.mu, '\n')
                if not converge:
                    break;
#                 else:
#                     continue
#                 break;
                    
#                     print('l:\n',l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau) - sum(self.lam * (self.beta) **2),'\n')

            score = 0
            predict = []
            self.beta = new_theta[:self.p].reshape(self.p, 1)
            for i in range(self.n):
#                     print('Lam:\n', self.lam, '\n')
#                     print('tau:', self.tau, '\n')
#                     print('Beta:\n', self.beta, '\n')
                score += l(self.X[i], self.y[i], self.mu[i], self.beta, self.tau) - sum(self.lam * (self.beta) **2)
                predict += [Pi(self.X[i], self.y[i], self.mu[i], self.beta, self.tau)]
            print('score:\n', score, '\n')
            if True in np.isnan(score):
                print('Error: NaN score')
                break;
            if (np.mean(score) > pre_score) and converge:
                optimized_beta = self.beta
                optimized_mu = self.mu
                optimized_lam = self.lam
                optimized_tau = self.tau
                # reset
                pre_score = score
                optimized_score = score
#                 self.tau = np.repeat(1.0, self.n)

        # Returning data
#         optimized_beta[0] = optimized_beta[0] + np.mean(optimized_tau * optimized_mu)
        self.beta = optimized_beta
        self.mu = optimized_mu
        self.lam = optimized_lam
        self.tau = optimized_tau
        self.score = optimized_score
        self.predict = np.concatenate(predict)


        X = np.concatenate(self.X)
        
        y = np.concatenate(self.y)

        V = np.diagflat(self.predict * (1 - self.predict) + 0.0001)

        SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(self.p,1)

        Z = self.beta/SE

        P = 2 * norm.cdf(-1 * np.abs(Z))

        CI_025  = self.beta - 1.959964 * SE
        CI_975  = self.beta + 1.959964 * SE

        self.df = pd.DataFrame({'Coef': np.transpose(self.beta)[0], 'Std.Err': np.transpose(SE)[0],
                           'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0],
                           '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                          index = self.var_name)
        
        return self

In [86]:
p = 10
np.random.seed(233)
true_beta = (np.random.rand(p,1) - np.random.rand(p,1)) * 10
true_tau = np.random.rand(1)
true_mu = np.random.rand(2)
X1 = (np.random.rand(1000, p) - np.random.rand(1000, p)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(true_mu[0], true_tau, 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, p) - np.random.rand(1000, p)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(true_mu[1], true_tau, 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)
X = [X1, X2]
y = [y1, y2]

In [87]:
# pd.DataFrame(X1).to_csv('/Users/wli17/Documents/GLMM/GLMM/X1_test.csv', header=None, index=None)
# pd.DataFrame(X2).to_csv('/Users/wli17/Documents/GLMM/GLMM/X2_test.csv', header=None, index=None)
# pd.DataFrame(y1).to_csv('/Users/wli17/Documents/GLMM/GLMM/y1_test.csv', header=None, index=None)
# pd.DataFrame(y2).to_csv('/Users/wli17/Documents/GLMM/GLMM/y2_test.csv', header=None, index=None)

In [88]:
true_tau

array([0.42244813])

In [89]:
true_mu

array([0.2513114 , 0.53939061])

In [90]:
model = LA(X, y).fit(lam_it=5)

In lambad = 0
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [0.42859166] 

In lambad = 1
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-48.19082457] 

In lambad = 2
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-65.01836753] 

In lambad = 3
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-76.48262169] 

In lambad = 4
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-85.42383709] 

In lambad = 5
The 1 step of mu
Done with iteration
The 2 step of mu
Done with iteration
score:
 [-92.87641947] 



In [93]:
model.df

,Coef,Std.Err,z,P-value,[0.025,0.975]
X1,-1.852312,0.046169,-40.120639,0.000000e+00,-1.942801,-1.761823
X2,4.455123,0.090925,48.997532,0.000000e+00,4.276913,4.633334
X3,-2.748092,0.061414,-44.746988,0.000000e+00,-2.868461,-2.627722
X4,-0.388153,0.030710,-12.639486,1.278871e-36,-0.448342,-0.327963
X5,2.487525,0.056635,43.922352,0.000000e+00,2.376523,2.598526
X6,-5.265764,0.106590,-49.402211,0.000000e+00,-5.474675,-5.056852
X7,-5.922779,0.118307,-50.062845,0.000000e+00,-6.154657,-5.690902
X8,-1.364450,0.039588,-34.466528,2.546482e-260,-1.442041,-1.286860
X9,-1.226655,0.037193,-32.981129,1.514630e-238,-1.299551,-1.153759
X10,-0.280893,0.029681,-9.463607,2.975004e-21,-0.339067,-0.222718
